In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import pandas as pd
import pickle
import numpy as np
import re


In [3]:
# 파일 불러오기
with open('../../data/df_tra.pkl', 'rb') as f:
    df_tra = pickle.load(f)


In [4]:
# 파일 불러오기
with open('../../data/df_tra_ans.pkl', 'rb') as f:
    df_tra_ans = pickle.load(f)


In [5]:
rel_path = '../LLM/result/'

In [6]:
df_tra.head()

,ver,q_id,u_id,q_creationdate,u_creationdate
0,50000,70230690,5685327,2021-12-05,2015-12-16 07:05:25.767
1,50000,70230749,16988273,2021-12-05,2021-09-23 20:54:47.807
2,50000,70231066,8434249,2021-12-05,2017-08-08 12:42:44.447
3,50000,70231081,3826634,2021-12-05,2014-07-10 17:56:10.877
4,50000,70232017,16369417,2021-12-05,2021-07-03 02:17:03.747


In [8]:
df_tra['q_creationdate'] = pd.to_datetime(df_tra['q_creationdate'])
df_tra['u_creationdate'] = pd.to_datetime(df_tra['u_creationdate'])


df_tra_ans['q_creationdate'] = pd.to_datetime(df_tra_ans['q_creationdate'])
df_tra_ans['a_u_creationdate'] = pd.to_datetime(df_tra_ans['a_u_creationdate'])

In [9]:
df_tra['diff_days'] = [x.days for x in df_tra['q_creationdate'] - df_tra['u_creationdate']]
df_tra_ans['diff_days'] = [x.days for x in df_tra_ans['q_creationdate'] - df_tra_ans['a_u_creationdate']]

In [10]:
df_tra.head()

,ver,q_id,u_id,q_creationdate,u_creationdate,diff_days
0,50000,70230690,5685327,2021-12-05,2015-12-16 07:05:25.767,2180
1,50000,70230749,16988273,2021-12-05,2021-09-23 20:54:47.807,72
2,50000,70231066,8434249,2021-12-05,2017-08-08 12:42:44.447,1579
3,50000,70231081,3826634,2021-12-05,2014-07-10 17:56:10.877,2704
4,50000,70232017,16369417,2021-12-05,2021-07-03 02:17:03.747,154


In [11]:
def make_one_file(ver):
    path = f'{rel_path}{ver}'
    file_list = os.listdir(path)
    df = pd.DataFrame()
    if len(file_list)>0 : 
        for f in file_list:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            df = pd.concat([df, tmp], axis =0)

        df.sort_values(by = ['creationdate']).reset_index(drop=True)
        return df
    else :
        return np.NAN

In [12]:
def pp_df(df, sc_num):
    df_copy = df.copy()
    df_copy['creationdate'] = pd.to_datetime(df_copy['creationdate'])
    df_c = df.copy()
    df_c = df_c[~df_c['result'].isna()]
    df_c['o_result'] = df_c['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
    df_c = df_c[df_c['o_result'].isin(['1', '0', '2'])]
    
    df_c.loc[:, 'cnt'] = 1
    chk_cnt = df_c.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'cnt']]
    chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]


    m_chk_cnt = pd.merge(chk_cnt, df_copy, on = 'id')

    m_chk_cnt = m_chk_cnt[['ver', 'creationdate', 'id', 'o_result']].drop_duplicates()
        
    return m_chk_cnt
        



In [13]:
path = f'{rel_path}'
file_list = os.listdir(path)
sample_list = sorted([y for y in [x for x in file_list if x.isdigit()] if int(y)>=50000])

In [14]:
tot_calc = pd.DataFrame()
for listid in sample_list:
    df = make_one_file(listid)
    if isinstance(df, pd.core.frame.DataFrame):
        df = pp_df(df, 5)
        tot_calc = pd.concat([tot_calc, df], axis = 0)

tot_calc = tot_calc.reset_index(drop=True)

    

In [15]:
df_merge = pd.merge(df_tra, tot_calc, left_on = ['ver', 'q_id', 'q_creationdate'], right_on = ['ver', 'id', 'creationdate'])


df_merge_ans = pd.merge(df_tra_ans, tot_calc, left_on = ['ver', 'q_id', 'q_creationdate'], right_on = ['ver', 'id', 'creationdate'])


In [16]:
# diff = pd.merge(tot_calc, df_merge[['ver', 'creationdate', 'id', 'u_id']], how = 'left', left_on = ['ver', 'creationdate', 'id'], right_on = ['ver', 'creationdate', 'id'])
# diff[diff['u_id'].isna()]
# users 테이블에  id 가 없는 경우 NaN으로 출력되는 경우 발생 (가입하지 않고도 글을 쓸 수 있나봄? 혹은 가입하고 글쓰고 나서 탈퇴하거나)

In [18]:
df_merge_ans.head()

,ver,q_id,a_u_id,q_creationdate,a_u_creationdate,diff_days,creationdate,id,o_result
0,200159,73793618,6865787,2022-09-20,2016-09-22 16:27:28.820,2188,2022-09-20,73793618,2
1,200034,73785055,18400908,2022-09-20,2022-03-07 17:59:01.410,196,2022-09-20,73785055,1
2,200084,73791428,459745,2022-09-20,2009-09-14 16:08:14.280,4753,2022-09-20,73791428,0
3,200034,73793403,5775112,2022-09-20,2016-01-11 18:34:28.027,2443,2022-09-20,73793403,1
4,50060,73966162,2607773,2022-10-05,2013-07-22 17:31:46.487,3361,2022-10-05,73966162,1


In [19]:
df_merge['signin'] = np.ceil(df_merge['diff_days']/365)
df_merge_ans['signin'] = np.ceil(df_merge_ans['diff_days']/365)

In [20]:
df_merge.head()

,ver,q_id,u_id,q_creationdate,u_creationdate,diff_days,creationdate,id,o_result,signin
0,50000,70231066,8434249,2021-12-05,2017-08-08 12:42:44.447,1579,2021-12-05,70231066,1,5.0
1,50000,70231081,3826634,2021-12-05,2014-07-10 17:56:10.877,2704,2021-12-05,70231081,1,8.0
2,50000,70232017,16369417,2021-12-05,2021-07-03 02:17:03.747,154,2021-12-05,70232017,1,1.0
3,50000,70232350,2711059,2021-12-05,2013-08-23 12:42:49.387,3025,2021-12-05,70232350,1,9.0
4,50000,70232524,13518757,2021-12-05,2020-05-11 14:46:00.823,572,2021-12-05,70232524,0,2.0


In [21]:
df_merge_ans.head()

,ver,q_id,a_u_id,q_creationdate,a_u_creationdate,diff_days,creationdate,id,o_result,signin
0,200159,73793618,6865787,2022-09-20,2016-09-22 16:27:28.820,2188,2022-09-20,73793618,2,6.0
1,200034,73785055,18400908,2022-09-20,2022-03-07 17:59:01.410,196,2022-09-20,73785055,1,1.0
2,200084,73791428,459745,2022-09-20,2009-09-14 16:08:14.280,4753,2022-09-20,73791428,0,14.0
3,200034,73793403,5775112,2022-09-20,2016-01-11 18:34:28.027,2443,2022-09-20,73793403,1,7.0
4,50060,73966162,2607773,2022-10-05,2013-07-22 17:31:46.487,3361,2022-10-05,73966162,1,10.0


In [22]:
# 전체 데이터셋 
with open('../../data/df_tra_user.pkl', 'wb') as f:
    pickle.dump(df_merge, f)

In [23]:
# 전체 데이터셋 
with open('../../data/df_tra_ans_user.pkl', 'wb') as f:
    pickle.dump(df_merge_ans, f)
